In [2]:
from siphon.catalog import TDSCatalog
import xarray as xr
import numpy as np
import pyproj

TARGET_LAT = 58.2238  # Ljungskile Latitude
TARGET_LON = 11.9224  # Ljungskile Longitude

In [43]:
latest = xr.open_dataset('/home/johan/Documents/PhD/PREMACOP/METNO-project/MEPS_latest_ljungskile/meps_lagged_6_h_latest_2_5km_20250813T16Z.nc')
latest

<xarray.Dataset> Size: 120kB
Dimensions:                    (time: 62, pressure: 2, ensemble_member: 30,
                                height0: 1, height2: 1, height1: 1,
                                height_above_msl: 1)
Coordinates:
  * time                       (time) datetime64[ns] 496B 2025-08-13T16:00:00...
  * ensemble_member            (ensemble_member) int16 60B 0 1 2 3 ... 27 28 29
  * pressure                   (pressure) float32 8B 850.0 925.0
  * height_above_msl           (height_above_msl) float32 4B 0.0
  * height0                    (height0) float32 4B 2.0
  * height1                    (height1) float32 4B 0.0
  * height2                    (height2) float32 4B 10.0
    x                          float32 4B ...
    y                          float32 4B ...
    longitude                  float64 8B ...
    latitude                   float64 8B ...
Data variables: (12/15)
    forecast_reference_time    datetime64[ns] 8B ...
    projection_lambert         int32 4B ...
    x_wind_pl                  (time, pressure, ensemble_member) float32 15kB ...
    y_wind_pl                  (time, pressure, ensemble_member) float32 15kB ...
    air_temperature_pl         (time, pressure, ensemble_member) float32 15kB ...
    air_temperature_2m         (time, height0, ensemble_member) float32 7kB ...
    ...                         ...
    cloud_area_fraction        (time, height1, ensemble_member) float32 7kB ...
    air_pressure_at_sea_level  (time, height_above_msl, ensemble_member) float32 7kB ...
    precipitation_amount_acc   (time, height1, ensemble_member) float32 7kB ...
    snowfall_amount_acc        (time, height1, ensemble_member) float32 7kB ...
    wind_speed_of_gust         (time, height2, ensemble_member) float32 7kB ...
    fog_area_fraction          (time, height1, ensemble_member) float32 7kB ...
Attributes: (12/15)
    Conventions:                     CF-1.6
    institution:                     Norwegian Meteorological Institute, MET ...
    creator_url:                     met.no
    source:                          MEPS 2.5km
    min_time:                        2025-08-16 04:00:00Z
    max_time:                        2025-08-16
    ...                              ...
    geospatial_lon_max:              54.2
    references:                      unknown
    license:                         https://www.met.no/en/free-meteorologica...
    comment:                         For more information, please visit https...
    history:                         2025-08-13T18:06:54 creation by fimex
    DODS_EXTRA.Unlimited_Dimension:  time

In [44]:
# URL for the THREDDS catalog
catalog_url = 'https://thredds.met.no/thredds/catalog/meps25epsarchive/2024/12/25/catalog.xml'
# catalog_url = 'https://thredds.met.no/thredds/catalog/mepslatest/catalog.xml'
catalog = TDSCatalog(catalog_url)
all_dataset_names = list(catalog.datasets)

In [45]:
search_string = 'meps_det_sfc'
filtered_list = [
        name for name in all_dataset_names 
        if search_string in name and name.endswith('.nc') and 'latest.nc' not in name
        ]

In [46]:
url = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2024/12/25/17/meps_mbr011_sfc_20241225T17Z.ncml'

# Open the remote dataset directly using OPeNDAP
ds = xr.open_dataset(url, engine="netcdf4")

# Use the more accurate projection-based method
crs_info = ds['projection_lambert'].attrs
meps_crs = pyproj.CRS.from_cf(
    {
        "grid_mapping_name": crs_info['grid_mapping_name'],
        "standard_parallel": crs_info['standard_parallel'],
        "longitude_of_central_meridian": crs_info['longitude_of_central_meridian'],
        "latitude_of_projection_origin": crs_info['latitude_of_projection_origin'],
        "earth_radius": crs_info['earth_radius'],
    }
)

proj = pyproj.Proj.from_crs(4326, meps_crs, always_xy=True)
X, Y = proj.transform(TARGET_LON, TARGET_LAT)

x_idx = np.argmin(np.abs(ds.x.values - X))
y_idx = np.argmin(np.abs(ds.y.values - Y))

# Select data using the integer indices
point_ds = ds.isel(y=y_idx, x=x_idx)

In [49]:
point_ds.data_vars

Data variables:
    forecast_reference_time                                   datetime64[ns] 8B ...
    projection_lambert                                        int32 4B ...
    cloud_base_altitude                                       (time, surface) float32 268B ...
    visibility_in_air                                         (time, height0) float32 268B ...
    air_temperature_0m                                        (time, height0) float32 268B ...
    surface_air_pressure                                      (time, height0) float32 268B ...
    integral_of_surface_net_downward_shortwave_flux_wrt_time  (time, height0) float32 268B ...
    specific_convective_available_potential_energy            (time, height0) float32 268B ...
    air_temperature_2m                                        (time, height1) float32 268B ...
    relative_humidity_2m                                      (time, height1) float32 268B ...
    x_wind_10m                                                (ti

In [54]:
common_vars = [var for var in latest.data_vars if var in point_ds.data_vars ]

In [55]:
common_vars

['forecast_reference_time',
 'projection_lambert',
 'air_temperature_2m',
 'relative_humidity_2m',
 'x_wind_10m',
 'y_wind_10m',
 'cloud_area_fraction',
 'air_pressure_at_sea_level',
 'precipitation_amount_acc',
 'snowfall_amount_acc',
 'wind_speed_of_gust',
 'fog_area_fraction']

In [50]:
latest.data_vars

Data variables:
    forecast_reference_time    datetime64[ns] 8B ...
    projection_lambert         int32 4B ...
    x_wind_pl                  (time, pressure, ensemble_member) float32 15kB ...
    y_wind_pl                  (time, pressure, ensemble_member) float32 15kB ...
    air_temperature_pl         (time, pressure, ensemble_member) float32 15kB ...
    air_temperature_2m         (time, height0, ensemble_member) float32 7kB ...
    relative_humidity_2m       (time, height0, ensemble_member) float32 7kB ...
    x_wind_10m                 (time, height2, ensemble_member) float32 7kB ...
    y_wind_10m                 (time, height2, ensemble_member) float32 7kB ...
    cloud_area_fraction        (time, height1, ensemble_member) float32 7kB ...
    air_pressure_at_sea_level  (time, height_above_msl, ensemble_member) float32 7kB ...
    precipitation_amount_acc   (time, height1, ensemble_member) float32 7kB ...
    snowfall_amount_acc        (time, height1, ensemble_member) float32 

In [20]:
# URL for the THREDDS catalog
catalog_url = 'https://thredds.met.no/thredds/catalog/nora3_subset_atmos/atm_hourly_v2/catalog.xml'
# catalog_url = 'https://thredds.met.no/thredds/catalog/mepslatest/catalog.xml'
catalog = TDSCatalog(catalog_url)
all_dataset_names = list(catalog.datasets)
BASE_URL = 'https://thredds.met.no/thredds/dodsC/nora3_subset_atmos/atm_hourly_v2'

In [21]:
filtered_list = [
        name for name in all_dataset_names if 'topo' not in name
        ]

In [22]:
filtered_list

['arome3km_1hr_202506.nc',
 'arome3km_1hr_202505.nc',
 'arome3km_1hr_202504.nc',
 'arome3km_1hr_202503.nc',
 'arome3km_1hr_202502.nc',
 'arome3km_1hr_202501.nc',
 'arome3km_1hr_202412.nc',
 'arome3km_1hr_202411.nc',
 'arome3km_1hr_202409.nc',
 'arome3km_1hr_202408.nc',
 'arome3km_1hr_202407.nc',
 'arome3km_1hr_202406.nc',
 'arome3km_1hr_202405.nc',
 'arome3km_1hr_202404.nc',
 'arome3km_1hr_202403.nc',
 'arome3km_1hr_202402.nc',
 'arome3km_1hr_202401.nc',
 'arome3km_1hr_202312.nc',
 'arome3km_1hr_202311.nc',
 'arome3km_1hr_202310.nc',
 'arome3km_1hr_202309.nc',
 'arome3km_1hr_202308.nc',
 'arome3km_1hr_202307.nc',
 'arome3km_1hr_202306.nc',
 'arome3km_1hr_202305.nc',
 'arome3km_1hr_202304.nc',
 'arome3km_1hr_202303.nc',
 'arome3km_1hr_202302.nc',
 'arome3km_1hr_202301.nc',
 'arome3km_1hr_202212.nc',
 'arome3km_1hr_202211.nc',
 'arome3km_1hr_202210.nc',
 'arome3km_1hr_202209.nc',
 'arome3km_1hr_202208.nc',
 'arome3km_1hr_202207.nc',
 'arome3km_1hr_202206.nc',
 'arome3km_1hr_202205.nc',
 

In [23]:
name = filtered_list[4]
url = f'{BASE_URL}/{name}'

# Open the remote dataset directly using OPeNDAP
ds = xr.open_dataset(url, engine="netcdf4")

# Use the more accurate projection-based method
crs_info = ds['projection_lambert'].attrs
meps_crs = pyproj.CRS.from_cf(
    {
        "grid_mapping_name": crs_info['grid_mapping_name'],
        "standard_parallel": crs_info['standard_parallel'],
        "longitude_of_central_meridian": crs_info['longitude_of_central_meridian'],
        "latitude_of_projection_origin": crs_info['latitude_of_projection_origin'],
        "earth_radius": crs_info['earth_radius'],
    }
)

proj = pyproj.Proj.from_crs(4326, meps_crs, always_xy=True)
X, Y = proj.transform(TARGET_LON, TARGET_LAT)

x_idx = np.argmin(np.abs(ds.x.values - X))
y_idx = np.argmin(np.abs(ds.y.values - Y))

# Select data using the integer indices
point_ds = ds.isel(y=y_idx, x=x_idx)

In [24]:
point_ds

<xarray.Dataset> Size: 40kB
Dimensions:                          (time: 672, height_above_msl: 1,
                                      height1: 1, height0: 1, height4: 1,
                                      height3: 1)
Coordinates:
  * height0                          (height0) float32 4B 0.0
  * height1                          (height1) float32 4B 2.0
  * height4                          (height4) float32 4B 10.0
  * height_above_msl                 (height_above_msl) float32 4B 0.0
  * time                             (time) datetime64[ns] 5kB 2025-02-01 ......
    x                                float32 4B 2.806e+06
    y                                float32 4B 3.885e+05
  * height3                          (height3) float32 4B 20.0
    latitude                         float64 8B ...
    longitude                        float64 8B ...
Data variables: (12/15)
    forecast_reference_time          datetime64[ns] 8B ...
    projection_lambert               int32 4B ...
    air_pressure_at_sea_level        (time, height_above_msl) float32 3kB ...
    air_temperature_2m               (time, height1) float32 3kB ...
    high_type_cloud_area_fraction    (time, height0) float32 3kB ...
    lifting_condensation_level       (time, height0) float32 3kB ...
    ...                               ...
    wind_direction                   (time, height4) float32 3kB ...
    wind_speed                       (time, height4) float32 3kB ...
    surface_net_longwave_radiation   (time, height0) float32 3kB ...
    surface_net_shortwave_radiation  (time, height0) float32 3kB ...
    precipitation_amount_hourly      (time, height0) float32 3kB ...
    fog                              (time, height3) float32 3kB ...
Attributes: (12/45)
    CDI:                             Climate Data Interface version 2.0.5 (ht...
    geospatial_lat_min:              44.
    geospatial_lat_max:              83.
    geospatial_lon_min:              -30.
    geospatial_lon_max:              85.
    comment:                         none
    ...                              ...
    time_coverage_start:             2025-02-01T00:00:00
    time_coverage_end:               2025-02-28T23:00:00
    processing_level:                scientific
    project:                         ERA4CS WINDSURFER
    spatial_representation:          grid
    DODS_EXTRA.Unlimited_Dimension:  time